In [17]:
from find_source import summary, get_info_for_catalog
import glob

In [18]:
for file in glob.glob('../data/250609_17:30:22/*.fits'):
    try:
        #summary(file, False, False, True, '../data/250609_17:30:22/figs_250609_17:30:22/')
        print(get_info_for_catalog(file))
    except:
        print(f'Try again for {file}')

Try again for ../data/250609_17:30:22/2232+117_full.fits
Try again for ../data/250609_17:30:22/2158-150_full.fits
{'name': '0423-013', 'obs_date_time': '6-10-25 9:38:57', 'file': '0423-013_full.fits', 'beam_maj_axis': <Angle 0.00062104 deg>, 'beam_min_axis': <Angle 0.00026885 deg>, 'beam_pos_angle': <Angle 71.03453 deg>, 'flux_uncertainty': 0.04948634281754494, 'source_1': {'flux_density': 0.8475363850593567, 'coord': <SkyCoord (ICRS): (ra, dec) in deg
    (65.8158375, 65.8158375)>, 'internal': True}}
{'name': '0136+478', 'obs_date_time': '6-10-25 9:38:49', 'file': '0136+478_full.fits', 'beam_maj_axis': <Angle 0.00039175 deg>, 'beam_min_axis': <Angle 0.00025615 deg>, 'beam_pos_angle': <Angle 84.62874 deg>, 'flux_uncertainty': 0.011283284053206444, 'source_1': {'flux_density': 1.849210262298584, 'coord': <SkyCoord (ICRS): (ra, dec) in deg
    (24.24408059, 24.24408635)>, 'internal': True}}
Try again for ../data/250609_17:30:22/1927+739_full.fits
{'name': 'neptune', 'obs_date_time': '6-1